# 상관분석(corelation analysis)
- 두 변수간의 관계의 정도를 알아보기 위한 방법
  - 공분산, 상관계수
- 공분산 => 2개 확률변수의 선형 관계를 나타내는 값, 음의 무한대~양의 무한대 값 가능
  - `COV(X, Y) = E[(X - E(X))(Y - E(Y))]`
- 상관계수 => 두 변수 사이의 상관 관계를 나타내는 값 [-1,1] 값 가능
  - 피어슨 상관계수(보통 상관계수라고 하면 이걸 의미), 스피어만 상관계수

# 회귀분석(regression analysis)
- 사이킷런 매뉴얼 참고 https://scikit-learn.org/0.23/
- 독립변수에 따른 종속변수의 변화를 분석하는 기법
  - 독립변수, 종속변수 모두 수치형 변수
- 단순회귀분석,다중회귀분석(다중공선성주의, 분산팽창요인 10 이상 확인),릿지회귀,라쏘회귀,엘라스틱넷
- 회귀분석 가정(아래 가정이 만족되어야 회귀분석이 의미 있음)
  - 독립/ 종속변수 선형성 확인 => 공분산, 산점도
  - 오차의 등분산성 => 독립변수에 따른 잔차 그래프
  - 오차의 독립성 => 오차간 관계없음, 더빈-왓슨 검정
  - 오차의 정규성 => QQ plot, KS 검정, shapro-wilk 검정
- 회귀분석 검토
  - 회귀계수 유의성 검정(T 검정통계량)
  - 모형 설명력 확인(결정계수, 수정결정계수)
  - 회귀모형 통계적 유의성 검정(F 검정통계량)
  - 다중회귀분석의 경우 다중공선성 확인 및 VIF(분산팽창요인) 10 이상 여부 확인
    - 분산팽창요인이 10 이상이면, 독립변수간 상관관계가 높다고 볼 수 있고, 모형의 설명력을 떨어뜨려 신뢰가 낮아지게됨
    - 보통 변수를 일부 선택하는 방식 등으로 해결


In [11]:
import pandas as pd
import numpy as np

x1 = [10,20,30,40,50]
x2 = [10,20,30,40,50]

# np의 공분산 함수 반환값은 공분산 행렬을 반환하게됨
np.cov(x1, x2)

# np의 상관계수 함수 반환값은 상관계수 행렬을 반환하게됨
# 즉, (0,0)과 (1,1)은 자기자신이고, 나머지가 실제 각 변수간 상관계수를 의미
np.corrcoef(x1,x2)

df1 = pd.DataFrame({'x1' : x1, 'x2' : x2})

# pd 역시 상관계수를 아래와 같이 계산할 수 있음
df1.corr()

,x1,x2
x1,1.0,1.0
x2,1.0,1.0


In [4]:
# 선형회귀분석
# R 코드와 다른 특징이 바로 요약 정보가 제공되는지 여부임
# R 코드는 보통 회귀분석수행하면 (lm함수로) 회귀 계수를 포함해서 결정계수, 수정결정계수, 모형 적합도 등을 수치값으로 계산해서 함께 보여줌
# 그러나 파이썬 코드는 그것을 개발자가 직접 수행 필요
# 사이킷런 문서 참고 https://scikit-learn.org/0.23/modules/linear_model.html#
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import metrics, model_selection
from sklearn.linear_model import LinearRegression

# 데이터 수집
# as_frame은 디폴트가 False이며, True로 해야만 data.frame로 pandas 데이터 프레임 추출 가능
# return_X_y값은 디폴트가 False이며, True로 해야만 입력, 종속변수를 파이썬 객체로 획득 가능
raw_data = datasets.load_diabetes(as_frame = True)

# 데이터 설명 확인
# print(data.DESCR)

# pandas 데이터프레임 획득
df_data = raw_data.data
df_target = raw_data.target

# 행과 열수 확인(index가 있으면 열수에서는 제외됨에 주의)
print(f"type={type(df_data)}, data shape={df_data.shape}, target shape={df_target.shape}")

# 데이터 일부 확인
print(df_data.head())

# 데이터 탐색 및 준비
# 단순 회귀 분석 수행 준비
# 홀드아웃 방식 데이터 분할 = train : test = 8 : 2
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
x_train, x_test, y_train, y_test = model_selection.train_test_split(
    df_data # 독립 변수
    , df_target # 종속 변수
    , train_size = 0.8 # 훈련 데이터 개수
    , test_size = 0.2 # 테스트 데이터 개수
)

print(f"x_train={len(x_train)}, x_test={len(x_test)}, y_train={len(y_train)}, y_test={len(y_test)}")

# 모델 훈련
linear_regressor = LinearRegression().fit(x_train, y_train)

# 단순회귀 모형의 회귀계수 및 절편 값 
print(f"coef={linear_regressor.coef_}, intercept={linear_regressor.intercept_}")

# 모델 성능 평가 - 1
# 단순히 regressor의 score 메서드를 호출해도 됨
print(f"train set performance={linear_regressor.score(x_train, y_train)}")
print(f"test set performance={linear_regressor.score(x_test, y_test)}")

# 모델 성능 평가 - 2
# predict를 직접 호출하고, 그 결과와 비교해도 됨
y_pred = linear_regressor.predict(x_test)
print(f"type={type(y_pred)}")

# 구축한 모형으로 예측한 값과 실제값(y_test)의 차이 확인
print(f"RMSE={metrics.mean_squared_error(y_true = y_test, y_pred = y_pred, squared = False):0.3f}")

# r2(결정계수, 상관계수의 제곱값), 1에 가까울 수록 모형이 관측값 예측을 잘한 것
print(f"R2={metrics.r2_score(y_true = y_test, y_pred = y_pred):0.3f}")

# 모델 성능 개선


type=<class 'pandas.core.frame.DataFrame'>, data shape=(442, 10), target shape=(442,)
        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005671 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6  
0 -0.002592  0.019908 -0.017646  
1 -0.039493 -0.068330 -0.092204  
2 -0.002592  0.002864 -0.025930  
3  0.034309  0.022692 -0.009362  
4 -0.002592 -0.031991 -0.046641  
x_train=353, x_test=89, y_train=353, y_test=89
coef=[    8.02744966  -271.78359176   504.8141465    376.01416511
 -1195.24696997   789.21589508   242.48781009   240.81412727
   838.95367307    72.75594037], intercept=154.57679145342019
train set performance=0

In [6]:
# 릿지회귀
# 가중치를 작게 만들어서 일반화시키도록 하는 회귀 방법
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import metrics, model_selection
from sklearn.linear_model import Ridge

# 데이터 수집
# as_frame은 디폴트가 False이며, True로 해야만 data.frame로 pandas 데이터 프레임 추출 가능
# return_X_y값은 디폴트가 False이며, True로 해야만 입력, 종속변수를 파이썬 객체로 획득 가능
raw_data = datasets.load_diabetes(as_frame = True)

# 데이터 설명 확인
# print(data.DESCR)

# pandas 데이터프레임 획득
df_data = raw_data.data
df_target = raw_data.target

# 행과 열수 확인(index가 있으면 열수에서는 제외됨에 주의)
print(f"type={type(df_data)}, data shape={df_data.shape}, target shape={df_target.shape}")

# 데이터 일부 확인
print(df_data.head())

# 데이터 탐색 및 준비
# 단순 회귀 분석 수행 준비
# 홀드아웃 방식 데이터 분할 = train : test = 8 : 2
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
x_train, x_test, y_train, y_test = model_selection.train_test_split(
    df_data # 독립 변수
    , df_target # 종속 변수
    , train_size = 0.8 # 훈련 데이터 개수
    , test_size = 0.2 # 테스트 데이터 개수
)

print(f"x_train={len(x_train)}, x_test={len(x_test)}, y_train={len(y_train)}, y_test={len(y_test)}")

# 모델 훈련
# 릿지 회귀는 알파값을 통해 가중치를 조정할 수 있음
# 알파값을 크게할수록 가중치가 작아짐
ridge_regressor = Ridge(alpha = 0.1).fit(x_train, y_train)

# 단순회귀 모형의 회귀계수 및 절편 값 
print(f"coef={ridge_regressor.coef_}, intercept={ridge_regressor.intercept_}")

# 모델 성능 평가 - 1
# 단순히 regressor의 score 메서드를 호출해도 됨
print(f"train set performance={ridge_regressor.score(x_train, y_train)}")
print(f"test set performance={ridge_regressor.score(x_test, y_test)}")

# 모델 성능 평가 - 2
# predict를 직접 호출하고, 그 결과와 비교해도 됨
y_pred = ridge_regressor.predict(x_test)
print(f"type={type(y_pred)}")

# 구축한 모형으로 예측한 값과 실제값(y_test)의 차이 확인
print(f"RMSE={metrics.mean_squared_error(y_true = y_test, y_pred = y_pred, squared = False):0.3f}")

# r2(결정계수, 상관계수의 제곱값), 1에 가까울 수록 모형이 관측값 예측을 잘한 것
print(f"R2={metrics.r2_score(y_true = y_test, y_pred = y_pred):0.3f}")

# 모델 성능 개선


type=<class 'pandas.core.frame.DataFrame'>, data shape=(442, 10), target shape=(442,)
        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005671 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6  
0 -0.002592  0.019908 -0.017646  
1 -0.039493 -0.068330 -0.092204  
2 -0.002592  0.002864 -0.025930  
3  0.034309  0.022692 -0.009362  
4 -0.002592 -0.031991 -0.046641  
x_train=353, x_test=89, y_train=353, y_test=89
coef=[ -17.91264848 -155.9418618   497.07256007  361.15898149  -61.01631421
  -93.05852103 -144.18682095  149.54707176  384.14803732  111.29039779], intercept=152.55424585886385
train set performance=0.5116932133

In [8]:
# 라쏘회귀
# 가중치를 작게 만들어서 일반화시키도록 하는 회귀 방법
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import metrics, model_selection
from sklearn.linear_model import Lasso

# 데이터 수집
# as_frame은 디폴트가 False이며, True로 해야만 data.frame로 pandas 데이터 프레임 추출 가능
# return_X_y값은 디폴트가 False이며, True로 해야만 입력, 종속변수를 파이썬 객체로 획득 가능
raw_data = datasets.load_diabetes(as_frame = True)

# 데이터 설명 확인
# print(data.DESCR)

# pandas 데이터프레임 획득
df_data = raw_data.data
df_target = raw_data.target

# 행과 열수 확인(index가 있으면 열수에서는 제외됨에 주의)
print(f"type={type(df_data)}, data shape={df_data.shape}, target shape={df_target.shape}")

# 데이터 일부 확인
print(df_data.head())

# 데이터 탐색 및 준비
# 단순 회귀 분석 수행 준비
# 홀드아웃 방식 데이터 분할 = train : test = 8 : 2
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
x_train, x_test, y_train, y_test = model_selection.train_test_split(
    df_data # 독립 변수
    , df_target # 종속 변수
    , train_size = 0.8 # 훈련 데이터 개수
    , test_size = 0.2 # 테스트 데이터 개수
)

print(f"x_train={len(x_train)}, x_test={len(x_test)}, y_train={len(y_train)}, y_test={len(y_test)}")

# 모델 훈련
# 라쏘 회귀는 알파값을 통해 가중치를 조정할 수 있음
# 알파값을 크게할수록 가중치가 작아짐
lasso_regressor = Lasso(alpha = 0.1, max_iter=50000).fit(x_train, y_train)

# 단순회귀 모형의 회귀계수 및 절편 값 
print(f"coef={lasso_regressor.coef_}, intercept={lasso_regressor.intercept_}")

# 모델 성능 평가 - 1
# 단순히 regressor의 score 메서드를 호출해도 됨
print(f"train set performance={lasso_regressor.score(x_train, y_train)}")
print(f"test set performance={lasso_regressor.score(x_test, y_test)}")

# 모델 성능 평가 - 2
# predict를 직접 호출하고, 그 결과와 비교해도 됨
y_pred = lasso_regressor.predict(x_test)
print(f"type={type(y_pred)}")

# 구축한 모형으로 예측한 값과 실제값(y_test)의 차이 확인
print(f"RMSE={metrics.mean_squared_error(y_true = y_test, y_pred = y_pred, squared = False):0.3f}")

# r2(결정계수, 상관계수의 제곱값), 1에 가까울 수록 모형이 관측값 예측을 잘한 것
print(f"R2={metrics.r2_score(y_true = y_test, y_pred = y_pred):0.3f}")

# 모델 성능 개선


type=<class 'pandas.core.frame.DataFrame'>, data shape=(442, 10), target shape=(442,)
        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005671 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6  
0 -0.002592  0.019908 -0.017646  
1 -0.039493 -0.068330 -0.092204  
2 -0.002592  0.002864 -0.025930  
3  0.034309  0.022692 -0.009362  
4 -0.002592 -0.031991 -0.046641  
x_train=353, x_test=89, y_train=353, y_test=89
coef=[  -0.         -167.8630042   543.00835509  283.94722826  -42.86390368
   -0.         -200.84332466    0.          516.62597691    7.85767261], intercept=153.70083251009163
train set performance=0.5376053723

In [10]:
# 엘라스틱넷
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import metrics, model_selection
from sklearn.linear_model import ElasticNet

# 데이터 수집
# as_frame은 디폴트가 False이며, True로 해야만 data.frame로 pandas 데이터 프레임 추출 가능
# return_X_y값은 디폴트가 False이며, True로 해야만 입력, 종속변수를 파이썬 객체로 획득 가능
raw_data = datasets.load_diabetes(as_frame = True)

# 데이터 설명 확인
# print(data.DESCR)

# pandas 데이터프레임 획득
df_data = raw_data.data
df_target = raw_data.target

# 행과 열수 확인(index가 있으면 열수에서는 제외됨에 주의)
print(f"type={type(df_data)}, data shape={df_data.shape}, target shape={df_target.shape}")

# 데이터 일부 확인
print(df_data.head())

# 데이터 탐색 및 준비
# 단순 회귀 분석 수행 준비
# 홀드아웃 방식 데이터 분할 = train : test = 8 : 2
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
x_train, x_test, y_train, y_test = model_selection.train_test_split(
    df_data # 독립 변수
    , df_target # 종속 변수
    , train_size = 0.8 # 훈련 데이터 개수
    , test_size = 0.2 # 테스트 데이터 개수
)

print(f"x_train={len(x_train)}, x_test={len(x_test)}, y_train={len(y_train)}, y_test={len(y_test)}")

# 모델 훈련
model = ElasticNet(alpha = 1.0, l1_ratio = 0.5).fit(x_train, y_train)

# 단순회귀 모형의 회귀계수 및 절편 값 
print(f"coef={model.coef_}, intercept={model.intercept_}")

# 모델 성능 평가 - 1
# 단순히 regressor의 score 메서드를 호출해도 됨
print(f"train set performance={model.score(x_train, y_train)}")
print(f"test set performance={model.score(x_test, y_test)}")

# 모델 성능 평가 - 2
# predict를 직접 호출하고, 그 결과와 비교해도 됨
y_pred = model.predict(x_test)
print(f"type={type(y_pred)}")

# 구축한 모형으로 예측한 값과 실제값(y_test)의 차이 확인
print(f"RMSE={metrics.mean_squared_error(y_true = y_test, y_pred = y_pred, squared = False):0.3f}")

# r2(결정계수, 상관계수의 제곱값), 1에 가까울 수록 모형이 관측값 예측을 잘한 것
print(f"R2={metrics.r2_score(y_true = y_test, y_pred = y_pred):0.3f}")

# 모델 성능 개선


type=<class 'pandas.core.frame.DataFrame'>, data shape=(442, 10), target shape=(442,)
        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005671 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6  
0 -0.002592  0.019908 -0.017646  
1 -0.039493 -0.068330 -0.092204  
2 -0.002592  0.002864 -0.025930  
3  0.034309  0.022692 -0.009362  
4 -0.002592 -0.031991 -0.046641  
x_train=353, x_test=89, y_train=353, y_test=89
coef=[ 0.07103956  0.          3.3273186   2.18203034  0.56802544  0.23333189
 -1.58361541  1.84388283  3.03496661  1.65122649], intercept=150.9488440663017
train set performance=0.00857395793742266
test set perf